## Import libraries

In [1]:
import pandas as pd
import numpy as np
import pyodbc

## Database Connection 

In [2]:
server = 'WIN-OE3JAUO9CMD' 
database = 'MaxAIML' 
username = 'MaxAIML' 
password = 'MaxAIML@#321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

## Importing data from database into pandas dataframes

In [3]:
query = "SELECT ITEM_CODE,Qty,DistCode FROM handheld_sales_master;"
order_df = pd.read_sql(query, cnxn)
print(order_df)

query1 = "SELECT DistCode,DistributorName FROM Outlet_Master;"
distributor_df = pd.read_sql(query1, cnxn)
print(distributor_df)

query3 = "SELECT item_code,item_descn,nob FROM mast_item where finished_product='Y';"
item_df = pd.read_sql(query3, cnxn)
print(item_df)

query4 = "SELECT DistCode,ItemId,StockQTY FROM Distributor_Stock where UpdateDate = (SELECT MAX(UpdateDate)FROM Distributor_Stock);"
cs_df = pd.read_sql(query4, cnxn)
print(cs_df)


       ITEM_CODE  Qty DistCode
0         030250  1.0   S21586
1         090255  1.0   S21586
2         010183  1.0   S39340
3         010201  2.0   S39340
4         030201  2.0   S39340
...          ...  ...      ...
631883    111001  1.0   S37271
631884    111001  1.0   S37271
631885    030501  1.0   P07654
631886    050501  2.0   P07654
631887    090129  2.0   P07654

[631888 rows x 3 columns]
      DistCode                           DistributorName
0       S37274                  SONALI TRADERS- BASIRHAT
1       J06550                 J. N. ENTERPRISE - CHAPRA
2       S21586                SUNIL CHANDRA DEY - LALBAG
3       N05779                      NON OPERATIVE DEALER
4       B00730            BABA LOKNATH AGENCY - JAGATPUR
...        ...                                       ...
38486   G05628  SETH STATIONERY-BHANDARHATI(GOODLUCK SOF
38487   B17340                 BHABANI ENTERPRISE - SURI
38488   A20646                ANUPAMA TRADERS - CHAKDAHA
38489   A10640                 

## Renaming columns

In [4]:
order_df = order_df.rename(columns={'ITEM_CODE':'item_code'}) 
order_df

,item_code,Qty,DistCode
0,030250,1.0,S21586
1,090255,1.0,S21586
2,010183,1.0,S39340
3,010201,2.0,S39340
4,030201,2.0,S39340
...,...,...,...
631883,111001,1.0,S37271
631884,111001,1.0,S37271
631885,030501,1.0,P07654
631886,050501,2.0,P07654


In [5]:
cs_df = cs_df.rename(columns={'ItemId':'item_code'}) 
cs_df

,DistCode,item_code,StockQTY
0,S35795,060501,2.0
1,P10915,032250,20.0
2,B09147,012251,2.0
3,H05044,051001,180.0
4,A20759,062250,20.0
...,...,...,...
34676,S19850,022251,1.0
34677,P06570,051001,16.0
34678,S19937,090600,5.0
34679,M14094,091200,68.0


## Merging order and item dataframes

In [6]:
order_item = pd.merge(order_df,item_df,on='item_code',how='inner')
order_item

,item_code,Qty,DistCode,item_descn,nob
0,030250,1.0,S21586,SP250ML PET ASP,28
1,030250,1.0,C04689,SP250ML PET ASP,28
2,030250,1.0,C04689,SP250ML PET ASP,28
3,030250,1.0,N00620,SP250ML PET ASP,28
4,030250,1.0,A17349,SP250ML PET ASP,28
...,...,...,...,...,...
631883,180130,4.0,T08330,MMAT125ML40P TP,40
631884,180130,5.0,S37269,MMAT125ML40P TP,40
631885,010301,5.0,S39801,CC300 ML CAN,24
631886,010301,5.0,S39801,CC300 ML CAN,24


## Merging order, item and distributor dataframes

In [7]:
order_item_distributor = pd.merge(order_item,distributor_df,on='DistCode',how='inner')
order_item_distributor

,item_code,Qty,DistCode,item_descn,nob,DistributorName
0,030250,1.0,S21586,SP250ML PET ASP,28,SUNIL CHANDRA DEY - LALBAG
1,030250,1.0,S21586,SP250ML PET ASP,28,SUNIL CHANDRA DEY - LALBAG
2,030250,1.0,S21586,SP250ML PET ASP,28,SUNIL CHANDRA DEY - LALBAG
3,030250,1.0,S21586,SP250ML PET ASP,28,SUNIL CHANDRA DEY - LALBAG
4,030250,1.0,S21586,SP250ML PET ASP,28,SUNIL CHANDRA DEY - LALBAG
...,...,...,...,...,...,...
57534308,032251,5.0,S42112,PSP2250ML(PET),9,SABUR ALI -BHARATPUR-II
57534309,032251,10.0,S42112,PSP2250ML(PET),9,SABUR ALI -BHARATPUR-II
57534310,032251,10.0,S42112,PSP2250ML(PET),9,SABUR ALI -BHARATPUR-II
57534311,032251,10.0,S42112,PSP2250ML(PET),9,SABUR ALI -BHARATPUR-II


## Merging distributor current stock and outlet dataframes

In [8]:
cs_distributor = pd.merge(cs_df,distributor_df,on='DistCode',how='inner')
cs_distributor

,DistCode,item_code,StockQTY,DistributorName
0,S35795,060501,2.00,SOMNATH HALDER-JIGRI MORE
1,S35795,090600,87.00,SOMNATH HALDER-JIGRI MORE
2,S35795,060250,1.00,SOMNATH HALDER-JIGRI MORE
3,S35795,090204,0.02,SOMNATH HALDER-JIGRI MORE
4,S35795,020201,1.00,SOMNATH HALDER-JIGRI MORE
...,...,...,...,...
360745,T04391,030250,23.01,PRADIP MAJUMDAR - SUTIA
360746,B12098,032250,5.00,BECHU DAS - PURBA KALUPUR
360747,B12098,032250,5.00,BECHU DAS - PURBA KALUPUR
360748,B12098,111001,1.00,BECHU DAS - PURBA KALUPUR


## Merging distributor current stock,outlet and item dataframes

In [9]:
cs_distributor_item = pd.merge(cs_distributor,item_df,on='item_code',how='inner')
cs_distributor_item

,DistCode,item_code,StockQTY,DistributorName,item_descn,nob
0,S35795,060501,2.0,SOMNATH HALDER-JIGRI MORE,LI500ML(PET),24
1,B09147,060501,14.0,BHABESH MONDAL - KAJI PARA,LI500ML(PET),24
2,B09147,060501,14.0,BHABESH MONDAL - KAJI PARA,LI500ML(PET),24
3,A20759,060501,1.0,A. ARMAN ENTERPRISE -TINPAKURIA,LI500ML(PET),24
4,A20759,060501,1.0,A. ARMAN ENTERPRISE -TINPAKURIA,LI500ML(PET),24
...,...,...,...,...,...,...
360745,S19883,022000,-5.0,SUJIT SHAOO - KAKHURDA,FA2LTR(PET),9
360746,M24607,022000,9.0,MD OBIDULLAHA-SULTANPUR,FA2LTR(PET),9
360747,P06586,010351,-1.0,PRABIR DEY - KIRNAHAR,CC CAN 350ML,24
360748,P06586,010351,-1.0,PRABIR DEY - KIRNAHAR,CC CAN 350ML,24


## Orders dataframe for a single distributor

In [10]:
for_each_distributor = order_item_distributor.loc[order_item_distributor['DistributorName'] == 'SARONGHI STORS - JAMBONI']
for_each_distributor = for_each_distributor.reset_index(drop=True)
for_each_distributor

,item_code,Qty,DistCode,item_descn,nob,DistributorName
0,030250,1.0,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
1,030250,1.0,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
2,030250,1.0,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
3,030250,1.0,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
4,030250,1.0,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
...,...,...,...,...,...,...
5233,091201,1.0,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI
5234,091201,1.0,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI
5235,091201,1.0,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI
5236,091201,1.0,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI


## Converting case to bottle for orders

In [11]:
nob_lists=[]
for row in for_each_distributor.itertuples():
    sql = """\
    DECLARE @RC int;
    EXEC @RC = [CaseToBtl] @QTY = ?, @NOB = ?;
    SELECT @RC AS rc;
    """
    values = (row.Qty,row.nob)
    cursor.execute(sql, values)
    rc = cursor.fetchval()
    nob_lists.append(rc)
    
for_each_distributor.insert(2, "Qty_in_nob", nob_lists, True)
for_each_distributor 

,item_code,Qty,Qty_in_nob,DistCode,item_descn,nob,DistributorName
0,030250,1.0,28,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
1,030250,1.0,28,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
2,030250,1.0,28,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
3,030250,1.0,28,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
4,030250,1.0,28,S19860,SP250ML PET ASP,28,SARONGHI STORS - JAMBONI
...,...,...,...,...,...,...,...
5233,091201,1.0,12,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI
5234,091201,1.0,12,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI
5235,091201,1.0,12,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI
5236,091201,1.0,12,S19860,PMZ1.2LTR (PET),12,SARONGHI STORS - JAMBONI


## Current stock dataframe for a single distributor

In [12]:
cs_for_each_distributor = cs_distributor_item.loc[cs_distributor_item['DistributorName'] == 'SARONGHI STORS - JAMBONI']
cs_for_each_distributor = cs_for_each_distributor.reset_index(drop=True)
cs_for_each_distributor

,DistCode,item_code,StockQTY,DistributorName,item_descn,nob
0,S19860,090600,113.0,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
1,S19860,090600,113.0,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
2,S19860,090600,113.0,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
3,S19860,090600,113.0,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
4,S19860,090600,113.0,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
...,...,...,...,...,...,...
454,S19860,090126,-4.0,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40
455,S19860,090126,-4.0,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40
456,S19860,090126,-4.0,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40
457,S19860,090126,-4.0,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40


## Converting case to bottle for current stock

In [13]:
nob_list=[]
for row in cs_for_each_distributor.itertuples():
    sql = """\
    DECLARE @RC int;
    EXEC @RC = [CaseToBtl] @QTY = ?, @NOB = ?;
    SELECT @RC AS rc;
    """
    values = (row.StockQTY,row.nob)
    cursor.execute(sql, values)
    rc = cursor.fetchval()
    nob_list.append(rc)
    
cs_for_each_distributor.insert(3, "StockQTY_in_nob", nob_list, True)
cs_for_each_distributor

,DistCode,item_code,StockQTY,StockQTY_in_nob,DistributorName,item_descn,nob
0,S19860,090600,113.0,2712,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
1,S19860,090600,113.0,2712,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
2,S19860,090600,113.0,2712,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
3,S19860,090600,113.0,2712,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
4,S19860,090600,113.0,2712,SARONGHI STORS - JAMBONI,MZ600ML (PET),24
...,...,...,...,...,...,...,...
454,S19860,090126,-4.0,-160,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40
455,S19860,090126,-4.0,-160,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40
456,S19860,090126,-4.0,-160,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40
457,S19860,090126,-4.0,-160,SARONGHI STORS - JAMBONI,MZ TP125 (40PK),40


## For each item calculating quantity for orders

In [14]:
item_qty_count = (for_each_distributor .
     groupby(['item_descn']).
     mean().
     reset_index()
     [['item_descn', 'Qty', 'Qty_in_nob']]
    )
item_qty_count['1.5xQty'] = (item_qty_count['Qty_in_nob']*1.5)
item_qty_count

,item_descn,Qty,Qty_in_nob,1.5xQty
0,CC180 ML CAN,1.000000,36.000000,54.000000
1,CC200ML RGB,1.166667,28.000000,42.000000
2,CC250ML PET ASP,1.000000,28.000000,42.000000
3,CC500ML(PET),1.000000,24.000000,36.000000
4,KWAT 1.0 PET,4.000000,60.000000,90.000000
5,KWAT 2.0 PET,2.333333,21.000000,31.500000
6,LI200ML RGB,1.000000,24.000000,36.000000
7,LI2250ML(PET),1.000000,9.000000,13.500000
8,LM250ML PET ASP,1.000000,28.000000,42.000000
9,MZ1.2LTR (PET),1.200000,14.400000,21.600000


## For each item calculating quantity for current stock

In [15]:
item_qty_count_cs = (cs_for_each_distributor .
     groupby(['item_descn']).
     mean().
     reset_index()
     [['item_descn', 'StockQTY', 'StockQTY_in_nob']]
    )
item_qty_count_cs

,item_descn,StockQTY,StockQTY_in_nob
0,CC180 ML CAN,4.30,174
1,CC200ML RGB,53.20,1292
2,CC250ML PET ASP,72.23,2039
3,CC300ML(RGB),3.00,72
4,CC500ML(PET),29.00,696
5,CC750ML(PET),7.00,168
6,D.COKE 180CAN,6.00,216
7,FA200ML RGB,3.00,72
8,FA2250ML(PET),12.00,108
9,FA250ML PET ASP,21.24,612


## Calculating Suggested Quantity

In [16]:
for i in item_qty_count['item_descn']:
    if (item_qty_count_cs['item_descn'] == i).any() :
        item_qty_count['Suggested_Qty_in_nob'] = (item_qty_count['1.5xQty'] - item_qty_count_cs['StockQTY_in_nob'])
            
item_qty_count

,item_descn,Qty,Qty_in_nob,1.5xQty,Suggested_Qty_in_nob
0,CC180 ML CAN,1.000000,36.000000,54.000000,-120.000000
1,CC200ML RGB,1.166667,28.000000,42.000000,-1250.000000
2,CC250ML PET ASP,1.000000,28.000000,42.000000,-1997.000000
3,CC500ML(PET),1.000000,24.000000,36.000000,-36.000000
4,KWAT 1.0 PET,4.000000,60.000000,90.000000,-606.000000
5,KWAT 2.0 PET,2.333333,21.000000,31.500000,-136.500000
6,LI200ML RGB,1.000000,24.000000,36.000000,-180.000000
7,LI2250ML(PET),1.000000,9.000000,13.500000,-58.500000
8,LM250ML PET ASP,1.000000,28.000000,42.000000,-66.000000
9,MZ1.2LTR (PET),1.200000,14.400000,21.600000,-590.400000


## Recommendations

In [18]:
recom_item_qty = item_qty_count.loc[item_qty_count['Suggested_Qty_in_nob']>0].round(2)
recom_item_qty = recom_item_qty.reset_index(drop=True)
recom_item_qty = pd.merge(recom_item_qty,item_df,on='item_descn',how='inner')
recom_item_qty

,item_descn,Qty,Qty_in_nob,1.5xQty,Suggested_Qty_in_nob,item_code,nob
0,MZF TP125(40PK),1.58,63.03,94.55,4.55,090129,40
1,SMZ1.75LT(PET),1.67,15.00,22.50,18.50,091752,9
2,SP1.25LTR(PET),1.00,12.00,18.00,178.00,031250,12
3,SP200ML RGB,1.57,37.71,56.57,74.57,030201,24


## Converting quantity from bottle to case

In [20]:
nob_lt=[]
for row in recom_item_qty.itertuples():
    sql = """\
    DECLARE @RC float;
    EXEC @RC = [QtyToBtlCase] @QTY = ?, @NOB = ?;
    SELECT @RC AS rc;
    """
    values = (row.Suggested_Qty_in_nob,row.nob)
    cursor.execute(sql, values)
    rc = cursor.fetchval()
    nob_lt.append(rc)
  
recom_item_qty.insert(5, "Suggested_Qty_in_case", nob_lt, True)
recom_item_qty['Qty_in_nob']=recom_item_qty['Qty_in_nob'].apply(np.ceil)
recom_item_qty['1.5xQty']=recom_item_qty['1.5xQty'].apply(np.ceil)
recom_item_qty['Suggested_Qty_in_nob']=recom_item_qty['Suggested_Qty_in_nob'].apply(np.ceil)
recom_item_qty = recom_item_qty.rename(columns={'Qty':'Average_Qty','Qty_in_nob':'Average_Qty_in_nob','1.5xQty':'Average_Qty_in_nobx1.5'})

In [21]:
print("Distributor => SARONGHI STORS - JAMBONI : Sales Recommendation\n")
print(recom_item_qty[['item_descn','Average_Qty','Average_Qty_in_nob','Average_Qty_in_nobx1.5','Suggested_Qty_in_nob','Suggested_Qty_in_case']])

Distributor => SARONGHI STORS - JAMBONI : Sales Recommendation

        item_descn  Average_Qty  Average_Qty_in_nob  Average_Qty_in_nobx1.5  \
0  MZF TP125(40PK)         1.58                64.0                    95.0   
1   SMZ1.75LT(PET)         1.67                15.0                    23.0   
2   SP1.25LTR(PET)         1.00                12.0                    18.0   
3      SP200ML RGB         1.57                38.0                    57.0   

   Suggested_Qty_in_nob  Suggested_Qty_in_case  
0                   5.0                   0.04  
1                  19.0                   2.00  
2                 178.0                  14.10  
3                  75.0                   3.02  
